In [1]:
%pip install 'chronos-forecasting>=2.0' 'pandas[pyarrow]' 'matplotlib' --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 120.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.2 MB/s eta 0:00:00


In [2]:
import os

# Use only 1 GPU if available
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ["HF_TOKEN"] =  ...

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from chronos import BaseChronosPipeline, Chronos2Pipeline
from IPython.display import clear_output

# Load the Chronos-2 pipeline
# GPU recommended for faster inference, but CPU is also supported
pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-2", 
    device_map="cuda"
        # device_map="cpu"
        )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/478M [00:00<?, ?B/s]

In [86]:
#add holidays as a feature and see if it improves performance
import pandas as pd
import holidays

def add_holiday_flags(
    df: pd.DataFrame,
    ts_col: str = "ds",
    local_tz: str = "America/Montreal",
    observed: bool = True,
    include_names: bool = False,
) -> pd.DataFrame:
    """
    Adds boolean columns:
      • is_qc_holiday       — Québec public holiday (CA-QC)
      • is_jewish_holiday   — Israeli public/Jewish holiday (IL)
    Optionally adds:
      • qc_holiday_name
      • jewish_holiday_name

    Notes:
      • Holiday checks are date-based (00:00–24:00 local calendar date),
        not sundown-to-sundown observance.
      • NaT timestamps are ignored gracefully.
    """
    out = df.copy()

    # 1) Parse to datetime
    out[ts_col] = pd.to_datetime(out[ts_col], errors="coerce")

    # 2) Get the calendar DATE to use for holiday lookup
    #    - If tz-aware: convert to Montreal then take .date
    #    - If naive: assume values already represent local Montreal wall-clock; just take .date
    if getattr(out[ts_col].dt, "tz", None) is not None:
        dates_for_calendar = out[ts_col].dt.tz_convert(local_tz).dt.date
    else:
        dates_for_calendar = out[ts_col].dt.date

    # 3) Build a SAFE integer year range for the holiday objects
    years_series = pd.Series(dates_for_calendar)
    years_series = years_series.dropna().map(lambda d: int(pd.Timestamp(d).year))
    if years_series.empty:
        raise ValueError("No valid datetimes found to extract holiday years.")
    years = list(range(int(years_series.min()), int(years_series.max()) + 1))

    # 4) Construct holiday calendars
    qc_holidays = holidays.Canada(subdiv="QC", years=years, observed=observed)
    il_holidays = holidays.Israel(years=years, observed=observed)

    # 5) Flag membership
    out["is_qc_holiday"] = [ ("yes" if d in qc_holidays else "no") if pd.notna(pd.Timestamp(d)) else "no"
                             for d in dates_for_calendar ]
    out["is_jewish_holiday"] = [ ("yes" if d in il_holidays else "no") if pd.notna(pd.Timestamp(d)) else "no"
                                 for d in dates_for_calendar ]

    if include_names:
        out["qc_holiday_name"] = [ qc_holidays.get(d, "") if pd.notna(pd.Timestamp(d)) else ""
                                   for d in dates_for_calendar ]
        out["jewish_holiday_name"] = [ il_holidays.get(d, "") if pd.notna(pd.Timestamp(d)) else ""
                                       for d in dates_for_calendar ]

    return out

df = pd.read_csv('https://www.dropbox.com/scl/fi/s83jig4zews1xz7vhezui/allDataWithCalculatedColumns.csv?rlkey=9mm4zwaugxyj2r4ooyd39y4nl&raw=1')
df.ds = pd.to_datetime(df.ds, errors="coerce")
df['id']='jgh'
df = add_holiday_flags(df, ts_col='ds', include_names=False)
df.tail()

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM,total_tbs,vert_tbs,pod_tbs,overflow,id
42863,2025-11-25 08:00:00,4,24,2,11,6,35,2,5,0,...,1,0,0,15,9,12,10,2,9,jgh
42864,2025-11-25 09:00:00,12,36,8,19,20,55,0,5,0,...,1,0,0,15,9,11,10,1,9,jgh
42865,2025-11-25 10:00:00,7,43,8,27,15,70,4,9,0,...,1,0,0,16,9,15,12,3,9,jgh
42866,2025-11-25 11:00:00,10,53,9,36,19,89,5,14,0,...,1,0,0,16,9,22,19,3,10,jgh
42867,2025-11-25 12:00:00,17,70,16,52,33,122,5,19,1,...,1,1,0,16,8,38,27,11,12,jgh


In [51]:
all_shifts_df = pd.read_csv('https://www.dropbox.com/scl/fi/yeyr2a7pj6nry8i2q3m0c/all_shifts.csv?rlkey=q1su2h8fqxfnlu7t1l2qe1w0q&raw=1')
all_shifts_df['shift_start'] = pd.to_datetime(all_shifts_df['shift_start']).dt.round('h')
all_shifts_df['shift_end'] = pd.to_datetime(all_shifts_df['shift_end']).dt.round('h')
all_shifts_df.head()

,scheduled_shift_id,group_id,user_id,employee_id,npi,first_name,last_name,facility_id,facility_ext_id,facility_name,...,shift_short_name,shift_start,shift_end,shift_hours,work_start,work_end,work_hours,count_as_shift,is_night,is_weekend
0,9021,1,40,NaN,NaN,Joel,Turner,1,NaN,Jewish General Hospital,...,W1,2021-01-01 08:00:00,2021-01-01 16:00:00,8.0,1/1/2021 08:00,1/1/2021 16:00,8.0,0,0,0
1,9022,1,52,NaN,NaN,Gregory,Marton,1,NaN,Jewish General Hospital,...,X1,2021-01-01 08:00:00,2021-01-01 17:00:00,9.0,1/1/2021 08:00,1/1/2021 17:00,9.0,1,0,0
2,9023,1,20,NaN,NaN,Will,Grad,1,NaN,Jewish General Hospital,...,X3,2021-01-01 08:00:00,2021-01-01 17:00:00,9.0,1/1/2021 08:00,1/1/2021 17:00,9.0,1,0,0
3,9025,1,17,NaN,NaN,Jerry,Dankoff,1,NaN,Jewish General Hospital,...,X4,2021-01-01 08:00:00,2021-01-01 16:00:00,8.0,1/1/2021 08:00,1/1/2021 16:00,8.0,1,0,0
4,9026,1,26,NaN,NaN,Marie Renee,Lajoie,1,NaN,Jewish General Hospital,...,X2,2021-01-01 08:00:00,2021-01-01 16:00:00,8.0,1/1/2021 08:00,1/1/2021 16:00,8.0,1,0,0


In [52]:
shift_types_dict = {'W1':'flow',
 'X1':'pod',
 'X3':'pod',
 'X4':'vertical',
 'X2':'vertical',
 'WOC1':'oncall',
 'WOC2':'oncall',
 'WOC3':'oncall',
 'X5':'pod',
 'W3':'overlap',
 'Y1':'pod',
 'Y3':'pod',
 'Y4':'vertical',
 'Y2':'vertical',
 'Y5':'pod',
 'Z1':'night',
 'Z2':'night',
 'D1':'pod',
 'R1':'pod',
 'P1':'vertical',
 'D2':'vertical',
 'OC1':'oncall',
 'OC2':'oncall',
 'V1':'flow',
 'A1':'pod',
 'G1':'vertical',
 'E1':'pod',
 'R2':'pod',
 'A2':'pod',
 'P2':'vertical',
 'E2':'vertical',
 'N1':'night',
 'N2':'night',
 'L2':'overlap',
 'L4':'overlap',
 'H1':'teaching',
 'B1':'vertical',
 'L1':'overlap',
 'W5':'overlap',
 'L6':'overlap',
 'B2':'vertical'}

In [53]:
all_shifts_df['shift_type'] = all_shifts_df['shift_short_name'].map(shift_types_dict)
all_shifts_df.head()

,scheduled_shift_id,group_id,user_id,employee_id,npi,first_name,last_name,facility_id,facility_ext_id,facility_name,...,shift_start,shift_end,shift_hours,work_start,work_end,work_hours,count_as_shift,is_night,is_weekend,shift_type
0,9021,1,40,NaN,NaN,Joel,Turner,1,NaN,Jewish General Hospital,...,2021-01-01 08:00:00,2021-01-01 16:00:00,8.0,1/1/2021 08:00,1/1/2021 16:00,8.0,0,0,0,flow
1,9022,1,52,NaN,NaN,Gregory,Marton,1,NaN,Jewish General Hospital,...,2021-01-01 08:00:00,2021-01-01 17:00:00,9.0,1/1/2021 08:00,1/1/2021 17:00,9.0,1,0,0,pod
2,9023,1,20,NaN,NaN,Will,Grad,1,NaN,Jewish General Hospital,...,2021-01-01 08:00:00,2021-01-01 17:00:00,9.0,1/1/2021 08:00,1/1/2021 17:00,9.0,1,0,0,pod
3,9025,1,17,NaN,NaN,Jerry,Dankoff,1,NaN,Jewish General Hospital,...,2021-01-01 08:00:00,2021-01-01 16:00:00,8.0,1/1/2021 08:00,1/1/2021 16:00,8.0,1,0,0,vertical
4,9026,1,26,NaN,NaN,Marie Renee,Lajoie,1,NaN,Jewish General Hospital,...,2021-01-01 08:00:00,2021-01-01 16:00:00,8.0,1/1/2021 08:00,1/1/2021 16:00,8.0,1,0,0,vertical


In [89]:
# Create hourly rows
# We'll use a list comprehension to generate the range for each row
expanded_rows = []
for _, row in all_shifts_df.iterrows():
    # Create range. inclusive='left' means [start, end)
    # If start == end (e.g. 0 length shift after rounding), it will be empty, which is correct
    hours = pd.date_range(row['shift_start'], row['shift_end'], freq='h', inclusive='left')
    for h in hours:
        expanded_rows.append({
            'ds': h,
            'user': row['first_name']+row['last_name'],
            'shift_type': row['shift_type'],
            'shift_short_name': row['shift_short_name']
        })

expanded_df = pd.DataFrame(expanded_rows)

# Pivot
# index=timestamp, columns=user_id, values=shift_type
hourly_shifts_by_user_df = expanded_df.pivot_table(
    index='ds', 
    columns='user', 
    values='shift_type', 
    aggfunc='first' # In case of duplicates, take the first
)

# Fill NaNs
hourly_shifts_by_user_df = hourly_shifts_by_user_df.fillna('NotWorking')
# hourly_shifts_by_user_df.reset_index(inplace=True)
hourly_shifts_by_user_df


user,AlanAzuelos,AlexGuttman,AlexanderHart,AlexandreDostaler,AlexisHaligua,AmelieBellemare,ArzuChaudhry,BernardUnger,DahliaGuttman,DanielMankarios,...,PaulBrisebois,PhilipStasiak,RafaelAroutiunian,SaraAhronheim,ShuoPeng,StephenRosenthal,TanLe,ThuHangTran,WayneChoi,WillGrad
ds,,,,,,,,,,,,,,,,,,,,,
2021-01-01 08:00:00,oncall,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,pod
2021-01-01 09:00:00,oncall,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,pod
2021-01-01 10:00:00,oncall,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,pod
2021-01-01 11:00:00,oncall,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,pod
2021-01-01 12:00:00,oncall,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,pod
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-03 03:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
2025-12-03 04:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
2025-12-03 05:00:00,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking


In [90]:
df = df.merge(hourly_shifts_by_user_df, on='ds')
df.tail()

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,PaulBrisebois,PhilipStasiak,RafaelAroutiunian,SaraAhronheim,ShuoPeng,StephenRosenthal,TanLe,ThuHangTran,WayneChoi,WillGrad
42856,2025-11-25 08:00:00,4,24,2,11,6,35,2,5,0,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,vertical
42857,2025-11-25 09:00:00,12,36,8,19,20,55,0,5,0,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,vertical
42858,2025-11-25 10:00:00,7,43,8,27,15,70,4,9,0,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,vertical
42859,2025-11-25 11:00:00,10,53,9,36,19,89,5,14,0,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,vertical
42860,2025-11-25 12:00:00,17,70,16,52,33,122,5,19,1,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,vertical


In [56]:
df.tail()

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,60,61,62,65,66,68,72,73,74,80
42935,2025-11-25 07:00:00,1.0,21.0,1.0,8.0,2.0,29.0,0.0,3.0,0.0,...,NotWorking,NotWorking,NotWorking,NotWorking,night,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
42936,2025-11-25 08:00:00,4.0,25.0,2.0,10.0,6.0,35.0,2.0,5.0,0.0,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
42937,2025-11-25 09:00:00,11.0,36.0,9.0,19.0,20.0,55.0,0.0,5.0,0.0,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
42938,2025-11-25 10:00:00,7.0,43.0,8.0,27.0,15.0,70.0,4.0,9.0,0.0,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking
42939,2025-11-25 11:00:00,11.0,54.0,8.0,35.0,19.0,89.0,5.0,14.0,0.0,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking


In [91]:
import pandas as pd
from pandas.tseries.frequencies import to_offset
from IPython.display import clear_output


ID_COL = "id"
TS_COL = "ds"
TARGETS = ['total_tbs']

df = df.copy()
df[TS_COL] = pd.to_datetime(df[TS_COL], errors="coerce")
df = df.dropna(subset=[TS_COL])

# Snap to exact hours (lowercase 'h' to avoid FutureWarning)
df[TS_COL] = df[TS_COL].dt.floor("h")

# Sort + dedupe
df = df.sort_values([ID_COL, TS_COL]).drop_duplicates([ID_COL, TS_COL], keep="last")

def regularize_hourly(g: pd.DataFrame) -> pd.DataFrame:
    """
    Reindex each group's timestamps to strict hourly and fill gaps.
    Works whether the grouping column is present or omitted (include_groups=False).
    """
    # The group key (id) is available as g.name; if ID_COL exists, prefer it.
    sid = g[ID_COL].iloc[0] if ID_COL in g.columns else g.name

    g = g.sort_values(TS_COL)
    full_idx = pd.date_range(g[TS_COL].min(), g[TS_COL].max(), freq="h")
    g = g.set_index(TS_COL).reindex(full_idx)
    g.index.name = TS_COL

    # restore id (constant for the whole group)
    g[ID_COL] = sid

    # numeric + fill for targets
    for col in TARGETS:
        if col in g.columns:
            g[col] = pd.to_numeric(g[col], errors="coerce").ffill().bfill()
    return g.reset_index()

# Call apply with include_groups=False if supported; else fall back
gb = df.groupby(ID_COL, group_keys=False)
try:
    df = gb.apply(regularize_hourly, include_groups=False)
except TypeError:
    # older pandas without include_groups
    df = gb.apply(regularize_hourly)

# Assert truly hourly (accept 'h' and 'H')
g = df[df[ID_COL] == "jgh"].sort_values(TS_COL)
freq = pd.infer_freq(g[TS_COL])
if not freq:
    raise ValueError("No inferable frequency after regularization.")
if to_offset(freq).name.lower() != "h":
    # extra check independent of infer_freq
    diffs = g[TS_COL].diff().dropna()
    bad = g.loc[diffs != pd.Timedelta(hours=1), TS_COL].head(10).tolist()
    raise ValueError(f"Non-1h gaps remain around: {bad}")



In [ ]:
hourly_shifts_by_user_df.reset_index()

RangeIndex(start=0, stop=43128, step=1)

In [98]:
future_df = hourly_shifts_by_user_df.reset_index()[hourly_shifts_by_user_df.reset_index()['ds'] > df['ds'].max()]
future_df['id'] = 'jgh'
future_df.head()

user,ds,AlanAzuelos,AlexGuttman,AlexanderHart,AlexandreDostaler,AlexisHaligua,AmelieBellemare,ArzuChaudhry,BernardUnger,DahliaGuttman,...,PhilipStasiak,RafaelAroutiunian,SaraAhronheim,ShuoPeng,StephenRosenthal,TanLe,ThuHangTran,WayneChoi,WillGrad,id
42941,2025-11-25 13:00:00,NotWorking,flow,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,vertical,jgh
42942,2025-11-25 14:00:00,NotWorking,flow,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,vertical,jgh
42943,2025-11-25 15:00:00,NotWorking,flow,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,vertical,jgh
42944,2025-11-25 16:00:00,NotWorking,pod,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,vertical,NotWorking,NotWorking,vertical,NotWorking,vertical,jgh
42945,2025-11-25 17:00:00,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,NotWorking,...,NotWorking,NotWorking,NotWorking,vertical,NotWorking,NotWorking,vertical,NotWorking,NotWorking,jgh


In [99]:
df.columns.tolist()

['ds',
 'INFLOW_STRETCHER',
 'Infl_Stretcher_cum',
 'INFLOW_AMBULATORY',
 'Infl_Ambulatory_cum',
 'Inflow_Total',
 'Inflow_Cum_Total',
 'INFLOW_AMBULANCES',
 'Infl_Ambulances_cum',
 'FLS',
 'CUM_ADMREQ',
 'CUM_BA1',
 'WAITINGADM',
 'TTStr',
 'TRG_HALLWAY1',
 'TRG_HALLWAY_TBS',
 'reoriented_cum',
 'reoriented_cum_MD',
 'QTRACK1',
 'RESUS',
 'Pod_T',
 'POD_GREEN',
 'POD_GREEN_TBS',
 'POD_YELLOW',
 'POD_YELLOW_TBS',
 'POD_ORANGE',
 'POD_ORANGE_TBS',
 'POD_CONS_MORE2H',
 'POD_IMCONS_MORE4H',
 'POD_XRAY_MORE2H',
 'POD_CT_MORE2H',
 'POST_POD1',
 'VERTSTRET',
 'RAZ_TBS',
 'RAZ_LAZYBOY',
 'RAZ_WAITINGREZ',
 'AMBVERT1',
 'AMBVERTTBS',
 'QTrack_TBS',
 'Garage_TBS',
 'RAZ_CONS_MORE2H',
 'RAZ_IMCONS_MORE4H',
 'RAZ_XRAY_MORE2H',
 'RAZ_CT_MORE2H1',
 'PSYCH1',
 'PSYCH_WAITINGADM',
 'total_tbs',
 'vert_tbs',
 'pod_tbs',
 'overflow',
 'AlanAzuelos',
 'AlexGuttman',
 'AlexanderHart',
 'AlexandreDostaler',
 'AlexisHaligua',
 'AmelieBellemare',
 'ArzuChaudhry',
 'BernardUnger',
 'DahliaGuttman',
 'DanielM

In [100]:
# Predict
pred_df = pipeline.predict_df(
    df,
    # future_df = future_df.head(24),
    prediction_length=24,
    quantile_levels=[0.1, 0.5, 0.9],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)
pred_df

,id,ds,target_name,predictions,0.1,0.5,0.9
0,jgh,2025-11-25 13:00:00,total_tbs,45.126152,39.901848,45.126152,49.848518
1,jgh,2025-11-25 14:00:00,total_tbs,46.989227,39.064972,46.989227,53.724674
2,jgh,2025-11-25 15:00:00,total_tbs,48.564598,39.395687,48.564598,56.618565
3,jgh,2025-11-25 16:00:00,total_tbs,51.027130,39.035091,51.027130,61.199917
4,jgh,2025-11-25 17:00:00,total_tbs,45.976257,32.280502,45.976257,57.228527
5,jgh,2025-11-25 18:00:00,total_tbs,42.747681,28.185560,42.747681,54.529945
6,jgh,2025-11-25 19:00:00,total_tbs,39.165573,25.021765,39.165573,52.172699
7,jgh,2025-11-25 20:00:00,total_tbs,35.816063,22.247152,35.816063,49.443329
8,jgh,2025-11-25 21:00:00,total_tbs,33.047218,19.866066,33.047218,46.139732
9,jgh,2025-11-25 22:00:00,total_tbs,28.108273,16.641737,28.108273,40.969330


In [101]:
# Predict
pred_df = pipeline.predict_df(
    df,
    future_df = future_df.head(24),
    prediction_length=24,
    quantile_levels=[0.1, 0.5, 0.9],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)
pred_df

,id,ds,target_name,predictions,0.1,0.5,0.9
0,jgh,2025-11-25 13:00:00,total_tbs,44.491661,39.787445,44.491661,49.224648
1,jgh,2025-11-25 14:00:00,total_tbs,46.719387,39.550117,46.719387,53.247261
2,jgh,2025-11-25 15:00:00,total_tbs,48.326118,39.218613,48.326118,56.325459
3,jgh,2025-11-25 16:00:00,total_tbs,51.372391,40.285080,51.372391,61.618084
4,jgh,2025-11-25 17:00:00,total_tbs,47.929153,34.762840,47.929153,59.780380
5,jgh,2025-11-25 18:00:00,total_tbs,44.449360,30.604620,44.449360,56.289330
6,jgh,2025-11-25 19:00:00,total_tbs,41.202785,27.633581,41.202785,54.105225
7,jgh,2025-11-25 20:00:00,total_tbs,37.426731,24.012003,37.426731,51.102440
8,jgh,2025-11-25 21:00:00,total_tbs,34.948315,21.796997,34.948315,47.979111
9,jgh,2025-11-25 22:00:00,total_tbs,30.457214,18.698462,30.457214,43.346039


In [102]:
df = pd.read_csv('https://www.dropbox.com/scl/fi/s83jig4zews1xz7vhezui/allDataWithCalculatedColumns.csv?rlkey=9mm4zwaugxyj2r4ooyd39y4nl&raw=1')
df.ds = pd.to_datetime(df.ds, errors="coerce")
df['id']='jgh'

ID_COL = "id"
TS_COL = "ds"
TARGETS = ['total_tbs']

df = df.copy()
df[TS_COL] = pd.to_datetime(df[TS_COL], errors="coerce")
df = df.dropna(subset=[TS_COL])

# Snap to exact hours (lowercase 'h' to avoid FutureWarning)
df[TS_COL] = df[TS_COL].dt.floor("h")

# Sort + dedupe
df = df.sort_values([ID_COL, TS_COL]).drop_duplicates([ID_COL, TS_COL], keep="last")

def regularize_hourly(g: pd.DataFrame) -> pd.DataFrame:
    """
    Reindex each group's timestamps to strict hourly and fill gaps.
    Works whether the grouping column is present or omitted (include_groups=False).
    """
    # The group key (id) is available as g.name; if ID_COL exists, prefer it.
    sid = g[ID_COL].iloc[0] if ID_COL in g.columns else g.name

    g = g.sort_values(TS_COL)
    full_idx = pd.date_range(g[TS_COL].min(), g[TS_COL].max(), freq="h")
    g = g.set_index(TS_COL).reindex(full_idx)
    g.index.name = TS_COL

    # restore id (constant for the whole group)
    g[ID_COL] = sid

    # numeric + fill for targets
    for col in TARGETS:
        if col in g.columns:
            g[col] = pd.to_numeric(g[col], errors="coerce").ffill().bfill()
    return g.reset_index()

# Call apply with include_groups=False if supported; else fall back
gb = df.groupby(ID_COL, group_keys=False)
try:
    df = gb.apply(regularize_hourly, include_groups=False)
except TypeError:
    # older pandas without include_groups
    df = gb.apply(regularize_hourly)

# Assert truly hourly (accept 'h' and 'H')
g = df[df[ID_COL] == "jgh"].sort_values(TS_COL)
freq = pd.infer_freq(g[TS_COL])
if not freq:
    raise ValueError("No inferable frequency after regularization.")
if to_offset(freq).name.lower() != "h":
    # extra check independent of infer_freq
    diffs = g[TS_COL].diff().dropna()
    bad = g.loc[diffs != pd.Timedelta(hours=1), TS_COL].head(10).tolist()
    raise ValueError(f"Non-1h gaps remain around: {bad}")

# Predict
pred_df = pipeline.predict_df(
    df,
    prediction_length=24,
    quantile_levels=[0.1, 0.5, 0.9],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)
pred_df


,id,ds,target_name,predictions,0.1,0.5,0.9
0,jgh,2025-11-25 13:00:00,total_tbs,45.377232,40.563789,45.377232,49.991749
1,jgh,2025-11-25 14:00:00,total_tbs,46.404762,39.218372,46.404762,52.918350
2,jgh,2025-11-25 15:00:00,total_tbs,48.138084,40.034172,48.138084,55.532391
3,jgh,2025-11-25 16:00:00,total_tbs,50.805550,40.232307,50.805550,60.013859
4,jgh,2025-11-25 17:00:00,total_tbs,45.450912,33.141747,45.450912,55.506058
5,jgh,2025-11-25 18:00:00,total_tbs,42.141335,28.823408,42.141335,53.107262
6,jgh,2025-11-25 19:00:00,total_tbs,39.071415,25.897091,39.071415,50.964432
7,jgh,2025-11-25 20:00:00,total_tbs,34.914925,22.325605,34.914925,47.248600
8,jgh,2025-11-25 21:00:00,total_tbs,32.484570,20.172312,32.484570,44.725822
9,jgh,2025-11-25 22:00:00,total_tbs,28.685661,17.493061,28.685661,40.842041
